<a href="https://colab.research.google.com/github/SONUXO/machine-learning/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D , GlobalAveragePooling2D
import itertools

In [ ]:
from os import listdir
from os.path import isdir, isfile, join

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications import InceptionV3
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
from zipfile import ZipFile
file_name= "archive.zip"

with ZipFile(file_name,"r") as zip:
  zip.extractall()
  print("done")

done


In [ ]:
classes = sorted(listdir("/content/Automobile-parts"))
classes

In [ ]:
pip install split-folders[full]

In [ ]:
import splitfolders
input_folder="Automobile-parts"
splitfolders.ratio(input_folder, output= "dataset",seed=42,
                   ratio=(.7,.1,.2),
                   group_prefix=None)

Copying files: 689 files [00:00, 4638.69 files/s]


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen= ImageDataGenerator(rescale=1./255, shear_range=0.5,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  rotation_range=40,
                                  brightness_range=[0.1,0.9]
                                  ).flow_from_directory("/content/dataset/train",
                                                                         target_size=(244,244),
                                                                         batch_size=20,
                                                                         class_mode="categorical")
                                  
test_datagen=ImageDataGenerator(rescale=1./255).flow_from_directory("/content/dataset/test",
                                                                    target_size=(244,244),
                                                                    batch_size=20,
                                                                    class_mode="categorical",
                                                                    shuffle=False)

valid_datagen= ImageDataGenerator(rescale=1./255, shear_range=0.5,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  rotation_range=40,
                                  brightness_range=[0.1,0.9]
                                  ).flow_from_directory("/content/dataset/val",
                                                                         target_size=(244,244),
                                                                         batch_size=20,
                                                                         class_mode="categorical")     

In [ ]:
tf.random.set_seed(42)
In_model = InceptionV3(include_top=False,input_shape=(244, 244,3))
In_model.trainable=False
model=tf.keras.Sequential([
    In_model,
    GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    Flatten(),     
    tf.keras.layers.Dense(units=1000,activation="relu"),
    tf.keras.layers.Dense(units=14,activation="softmax")
  ]
)

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=["accuracy"]
              )

In [ ]:
history=model.fit(x=train_datagen, validation_data=valid_datagen,epochs=20, verbose=1 , callbacks= [tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=10, mode='min')]
                 )

In [ ]:
def make_confusion_matrix(con_m, classes, normalize=False, title ="confusion matrix",cmap=plt.cm.Blues):
  plt.imshow(con_m, interpolation="nearest", cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks= np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=90)
  plt.yticks(tick_marks, classes)

  if normalize:
    con_m=con_m.astype("float")/cm.sum(axis=1)[:,np.newaxis]
    print("Normalized confusion matrix")
  else:
    print("cofusion. matrix, without normalization")

  print(con_m)

  thresh= con_m.max()/2

  for i, j in itertools.product(range(con_m.shape[0]), range(con_m.shape[1])):
    plt.text(j,i,con_m[i,j],horizontalalignment="center",color="white" if con_m[i,j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel("True label")
  plt.xlabel("predicted label")

In [ ]:
predictions=model.predict(x=test_datagen,verbose=0)
results=np.round(predictions)
results

In [ ]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_true=test_datagen.classes, y_pred=np.argmax(predictions, axis=-1))
make_confusion_matrix(con_m=cm,classes=classes,title="confusion_matrix")

In [ ]:
loss, accuracy = model.evaluate(valid_datagen)
print('test accuracy : ', accuracy)
print('loss :', loss)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=test_datagen.classes, y_pred=np.argmax(predictions, axis=-1)))

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
from tensorflow.keras.preprocessing import image
img = image.load_img("/content/photo.jpg", target_size=(244, 244))
plt.imshow(img)
plt.show() 

In [ ]:
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

In [ ]:
test1=model.predict(img_batch,verbose=0)
results=np.round(test1)
results